In [133]:
import csv
import os
import sys
import numpy as np
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

def csv_reader_gaze(filePath):
    
    path = filePath
    #hash table
    fixations = []
    duration = []
    count = 0
    fixations_seq = []
    duration_seq = []
    subjects_seq = []
    length_seq = []
    block_seq = []
    image_seq = []
    
    res = {}
    cnt = 0
    sub = '126'
    img = '43'
    blk = 'dash'

    with open(path) as f:
        f_csv = csv.DictReader(f)
        
        for row in f_csv:
            count += 1
            if((row['Subject']==sub) & (row['Image']==img) & (row['Block']==blk)):
                cnt += 1
                fixations.append((float(row['X']),float(row['Y'])))
                duration.append(int(row['Duration']))
                if(count == 80212):
                    fixations_seq.append(fixations)
                    duration_seq.append(duration)
                    subjects_seq.append(sub)
                    length_seq.append(cnt)
                    block_seq.append(blk)
                    image_seq.append(int(img))
                    #res = {'subject':subjects_seq, 'block':block_seq, 'image':image_seq,
                           #'fixations':fixations_seq , 'duration':duration_seq, 'length':length_seq, }
            else:           
                fixations_seq.append(fixations)
                duration_seq.append(duration)
                subjects_seq.append(sub)
                length_seq.append(cnt)
                block_seq.append(blk)
                image_seq.append(int(img))
                #res = {'subject':subjects_seq, 'block':block_seq, 'image':image_seq,
                       #'fixations':fixations_seq , 'duration':duration_seq, 'length':length_seq, }
                fixations = []
                duration = []
                fixations.append((float(row['X']),float(row['Y'])))
                duration.append(int(row['Duration']))
                sub = row['Subject']
                img = row['Image']
                blk = row['Block']
                cnt = 1
    # Here, the dictionary variable res is kind of a pointer(address), when it is passed to another variable
    # the address is copied instead of the contents
    res = {'Subject':subjects_seq, 'Block':block_seq, 'ImgNumber':image_seq,
           'Fixations':fixations_seq , 'Duration':duration_seq, 'Length':length_seq}
    return res

def csv_reader_img(filepath):
    
    path = filepath
    block = []
    imgNumber = []
    location = []
    res = {}
    
    with open(path) as f:
        f_csv = csv.DictReader(f)
        for row in f_csv:
            block.append(row['Block'])
            imgNumber.append(int(row['ImgNumber']))
            sixteenth_x = ((float(row['axMax']) - float(row['axMin']))/16.0)
            sixteenth_y = ((float(row['ayMax']) - float(row['ayMin']))/16.0)
            location.append(((float(row['axMin'])),(float(row['axMax'])),(float(row['ayMin'])),(float(row['ayMax'])),
                            (float(row['bxMin'])),(float(row['bxMax'])),(float(row['byMin'])),(float(row['byMax'])),
                            (sixteenth_x), (sixteenth_y)))
                                
    res = {'Block':block, 'ImgNumber':imgNumber, 'Location':location}
    #print (res['Location'])
    return res

def gaze_to_fmap (csv_gaze, csv_imgDim):
    
    cnt = 0
    cnt1 = 80
    x = 0
    y = 0
    fmap_seq = []
    # iterate fixation sequences
    for i in range(len(csv_gaze['Fixations'])):
        #for dash images
        if csv_gaze['Block'][i] == 'dash':
            # iterate fixations for a single subject and a single image
            fmap = []
            for j in range(csv_gaze['Length'][i]):
                # determine the fixation to be in imgage a or b
                # in image a
                if ((csv_gaze['Fixations'][i][j][0] > csv_imgDim['Location'][cnt][0]) &
                ((csv_gaze['Fixations'][i][j][0] < csv_imgDim['Location'][cnt][1])) &
                ((csv_gaze['Fixations'][i][j][1] > csv_imgDim['Location'][cnt][2])) &
                ((csv_gaze['Fixations'][i][j][1] < csv_imgDim['Location'][cnt][3])) ):
                    # compute how many units (sixteenth) does this fixation consist
                    x = int(np.floor((csv_gaze['Fixations'][i][j][0] - csv_imgDim['Location'][cnt][0]) / 
                                csv_imgDim['Location'][cnt][8] + 0.5))
                    y = int(np.floor((csv_gaze['Fixations'][i][j][1] - csv_imgDim['Location'][cnt][2]) / 
                                csv_imgDim['Location'][cnt][9] + 0.5))
                    fmap.append((x, y))
                # in image b
                elif ((csv_gaze['Fixations'][i][j][0] > csv_imgDim['Location'][cnt][4]) &
                ((csv_gaze['Fixations'][i][j][0] < csv_imgDim['Location'][cnt][5])) &
                ((csv_gaze['Fixations'][i][j][1] > csv_imgDim['Location'][cnt][6])) &
                ((csv_gaze['Fixations'][i][j][1] < csv_imgDim['Location'][cnt][7])) ):

                    x = int(np.floor((csv_gaze['Fixations'][i][j][0] - csv_imgDim['Location'][cnt][4]) /
                                csv_imgDim['Location'][cnt][8] + 0.5) + 16)
                    y = int(np.floor((csv_gaze['Fixations'][i][j][1] - csv_imgDim['Location'][cnt][6]) / 
                                csv_imgDim['Location'][cnt][9] + 0.5))
                    fmap.append((x, y))
                    
            fmap_seq.append(fmap)        
            cnt += 1
            if cnt >= 79:
                cnt = 0
        #for ball images
        elif csv_gaze['Block'][i] == 'ball':
            # iterate fixations for a single subject and a single image
            fmap = []
            for j in range(csv_gaze['Length'][i]):
                # determine the fixation to be in imgage a or b
                # in image a
                if ((csv_gaze['Fixations'][i][j][0] > csv_imgDim['Location'][cnt][0]) &
                ((csv_gaze['Fixations'][i][j][0] < csv_imgDim['Location'][cnt][1])) &
                ((csv_gaze['Fixations'][i][j][1] > csv_imgDim['Location'][cnt][2])) &
                ((csv_gaze['Fixations'][i][j][1] < csv_imgDim['Location'][cnt][3])) ):
                    # compute how many units (sixteenth) does this fixation consist of
                    x = int(np.floor((csv_gaze['Fixations'][i][j][0] - csv_imgDim['Location'][cnt][0]) / 
                                csv_imgDim['Location'][cnt][8] + 0.5))
                    y = int(np.floor((csv_gaze['Fixations'][i][j][1] - csv_imgDim['Location'][cnt][2]) / 
                                csv_imgDim['Location'][cnt][9] + 0.5))
                    fmap.append((x, y))
                # in image b
                elif ((csv_gaze['Fixations'][i][j][0] > csv_imgDim['Location'][cnt][4]) &
                ((csv_gaze['Fixations'][i][j][0] < csv_imgDim['Location'][cnt][5])) &
                ((csv_gaze['Fixations'][i][j][1] > csv_imgDim['Location'][cnt][6])) &
                ((csv_gaze['Fixations'][i][j][1] < csv_imgDim['Location'][cnt][7])) ):

                    x = int(np.floor((csv_gaze['Fixations'][i][j][0] - csv_imgDim['Location'][cnt][4]) /
                                csv_imgDim['Location'][cnt][8] + 0.5) + 16)
                    y = int(np.floor((csv_gaze['Fixations'][i][j][1] - csv_imgDim['Location'][cnt][6]) / 
                                csv_imgDim['Location'][cnt][9] + 0.5))
                    fmap.append((x, y))
                    
            fmap_seq.append(fmap)        
            cnt += 1
            if cnt >= 159:
                cnt = 80
        # error                
        else:
            print('----error----: unsurported format')
    fmap_dic = {'Fmap': fmap_seq}        
    csv_gaze.update(fmap_dic)
    return csv_gaze

In [142]:
fp_gaze = './overallStats.csv'
fp_imgDim = './imgDim.csv'
csv_gaze = csv_reader_gaze(fp_gaze)
csv_imgDim = csv_reader_img(fp_imgDim)
gaze_to_fmap(csv_gaze, csv_imgDim)
m = 80


print(csv_imgDim['Block'][m], csv_imgDim['ImgNumber'][m], csv_imgDim['Location'][m])
#print (csv_gaze['fixations'][0][0])
print(csv_gaze['Subject'][m], csv_gaze['Block'][m], csv_gaze['ImgNumber'][m],
      csv_gaze['Length'][m], csv_gaze['Fixations'][m], csv_gaze['Fmap'][m])

x_pos = []
y_pos = []
for i in range(len(csv_gaze['Fixations'][m])):
    x_pos.append(csv_gaze['Fixations'][m][i][0])
    y_pos.append(csv_gaze['Fixations'][m][i][1])
print x_pos
print y_pos

x1_pos = []
y1_pos = []
for i in range(len(csv_gaze['Fmap'][m])):
    x1_pos.append(csv_gaze['Fmap'][m][i][0])
    y1_pos.append(csv_gaze['Fmap'][m][i][1])
print x1_pos
print y1_pos


('ball', 32, (150.32, 689.68, 402.16, 647.84, 990.95, 1529.05, 402.16, 647.84, 33.709999999999994, 15.355))
('116', 'dash', 32, 50, [(827.7, 550.34), (405.68, 556.98), (597.48, 534.73), (1203.81, 530.98), (1325.52, 535.12), (756.6, 536.73), (395.12, 554.46), (398.84, 552.94), (845.11, 530.69), (1183.8, 544.67), (1185.35, 545.86), (1343.44, 553.64), (585.02, 554.29), (1187.19, 550.85), (1190.55, 543.4), (1203.52, 577.07), (1326.79, 543.04), (787.43, 551.36), (396.03, 572.79), (1282.08, 542.47), (1284.0, 544.0), (752.19, 541.99), (447.67, 561.6), (974.66, 543.06), (1247.65, 550.41), (813.66, 544.55), (489.12, 564.54), (343.38, 578.5), (343.93, 559.27), (343.3, 557.8), (914.18, 532.59), (1269.23, 538.43), (1333.3, 544.16), (1333.54, 543.38), (1038.43, 525.85), (1365.67, 547.95), (1340.53, 546.51), (723.95, 538.25), (343.42, 568.96), (315.82, 550.32), (1281.87, 527.35), (1330.59, 536.64), (753.5, 529.92), (404.69, 541.38), (325.16, 549.33), (444.1, 565.71), (1211.56, 571.42), (1322.51, 554

In [143]:
#plotly.tools.set_credentials_file(username='shaohua_jia', api_key='5lrSmQvXQdt2yzfLYebg')
#display resolution : 1680X1050

trace0 = go.Scatter(
    x=x_pos,
    y=y_pos,
    mode='markers'
)

trace1 = go.Scatter(
    x=x1_pos,
    y=y1_pos,
    mode='markers'
)

fig = tools.make_subplots(rows=2, cols=1, vertical_spacing=0.001)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 2, 1)                          

fig['layout']['xaxis1'].update(range=[0, 1680])
fig['layout']['xaxis2'].update(range=[0, 38])
fig['layout']['yaxis1'].update(title='row', range=[1050, 0])
fig['layout']['yaxis2'].update(title='fmap row', range=[16, 0])                          
fig['layout'].update(height=800, width=1200, title='fixations to fmap')

# image a
fig['layout']['shapes'].append(                                
        dict(    
            type = 'rect',
            xref = 'x',
            yref = 'y',
            x0 = 296.45,
            y0 = 627.25,
            x1 = 543.55,
            y1 = 422.75,
            line = dict(
                color =  'rgba(128, 0, 128, 1)',
            ),
        )
)
# image b
fig['layout']['shapes'].append(                                 
        dict(    
            type = 'rect',
            xref = 'x',
            yref = 'y',
            x0 = 1136.45,
            y0 = 627.25,
            x1 = 1383.55,
            y1 = 422.75,
            line = dict(
                color =  'rgba(128, 0, 128, 1)',
            ),
        )
)
# fmap a
fig['layout']['shapes'].append(                                
        dict(    
            type = 'rect',
            xref = 'x2',
            yref = 'y2',
            x0 = 1,
            y0 = 16,
            x1 = 16,
            y1 = 1,
            line = dict(
                color =  'rgba(0, 128, 128, 1)',
            ),
        )
)
# fmap b
fig['layout']['shapes'].append(                                
        dict(    
            type = 'rect',
            xref = 'x2',
            yref = 'y2',
            x0 = 16,
            y0 = 16,
            x1 = 32,
            y1 = 1,
            line = dict(
                color =  'rgba(0, 128, 128, 1)',
            ),
        )
)
    
py.iplot(fig, filename='LENS_pre')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]

